In [6]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [7]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'df_OHLCV_{date_str}_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-03
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-03.parquet
dest_path: ..\data\df_OHLCV_2025-04-03_clean.parquet


In [8]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open    High     Low   Close  Adj Close   Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                       
MNDY   2025-04-03  251.00  253.18  233.80  233.63     233.63  1229809    251.00    253.18   233.80
       2025-04-02  250.99  269.82  250.99  266.60     266.60   644600    250.99    269.82   250.99
       2025-04-01  243.87  263.80  240.73  262.18     262.18  1361500    243.87    263.80   240.73
       2025-03-31  236.51  245.32  230.66  243.16     243.16   680000    236.51    245.32   230.66
       2025-03-28  255.05  255.45  244.30  247.01     247.01   618300    255.05    255.45   244.30

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 389295 entries, ('MNDY', Timestamp('2025-04-03 00:00:00')) to ('TRP', Timestamp('2024-04-04 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       389295 non-null  float64
 1   High       389295 non-null  float64
 2   Low        389295 non-null  float64
 3   Close      389295 non-null  float64
 4   Adj Close  389295 non-null  float64
 5   Volume     389274 non-null  Int64  
 6   Adj Open   389295 non-null  float64
 7   Adj High   389295 non-null  float64
 8   Adj Low    389295 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.6+ MB


In [9]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Original number of symbols: 1559
Number of symbols after filtering: 1538
Number of symbols filtered out: 21

First 10 symbols that were filtered out:
['ALTR', 'STRK', 'ITCI', 'ZK', 'SFD', 'SW', 'TLX', 'SAIL', 'CRWV', 'ULS']

Example of dates for first filtered out symbol:

Dates for ALTR:
DatetimeIndex(['2025-03-26', '2025-03-25', '2025-03-24', '2025-03-21', '2025-03-20', '2025-03-19', '2025-03-18', '2025-03-17', '2025-03-14', '2025-03-13',
               ...
               '2024-04-17', '2024-04-16', '2024-04-15', '2024-04-12', '2024-04-11', '2024-04-10', '2024-04-09', '2024-04-08', '2024-04-05', '2024-04-04'], dtype='datetime64[ns]', name='Date', length=245, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('MNDY', Timestamp('2025-04-03 00:00:00')) to ('TRP', Timestamp('2024-04-04 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-

In [10]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\df_OHLCV_2025-04-03_clean.parquet
